In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
file_path = '/content/drive/MyDrive/LSTM Chatbot/Conversation.csv'  # Your file path here
df = pd.read_csv(file_path)
print(df.head())

   Unnamed: 0                             question  \
0           0               hi, how are you doing?   
1           1        i'm fine. how about yourself?   
2           2  i'm pretty good. thanks for asking.   
3           3    no problem. so how have you been?   
4           4     i've been great. what about you?   

                                     answer  
0             i'm fine. how about yourself?  
1       i'm pretty good. thanks for asking.  
2         no problem. so how have you been?  
3          i've been great. what about you?  
4  i've been good. i'm in school right now.  


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Extract questions and answers as lists
questions = df['question'].astype(str).tolist()
answers = df['answer'].astype(str).tolist()

# Add <start> and <end> tokens to answers for training decoder
answers = ['<start> ' + ans + ' <end>' for ans in answers]

# Create tokenizers for questions and answers
tokenizer_questions = Tokenizer(filters='')
tokenizer_answers = Tokenizer(filters='')

# Fit tokenizers on texts
tokenizer_questions.fit_on_texts(questions)
tokenizer_answers.fit_on_texts(answers)

# Convert questions and answers to sequences of tokens
questions_seq = tokenizer_questions.texts_to_sequences(questions)
answers_seq = tokenizer_answers.texts_to_sequences(answers)

# Pad sequences to the maximum length in respective sets
max_len_questions = max(len(seq) for seq in questions_seq)
max_len_answers = max(len(seq) for seq in answers_seq)

questions_pad = pad_sequences(questions_seq, maxlen=max_len_questions, padding='post')
answers_pad = pad_sequences(answers_seq, maxlen=max_len_answers, padding='post')

print(f'Max question length: {max_len_questions}')
print(f'Max answer length: {max_len_answers}')


Max question length: 19
Max answer length: 21


In [4]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Prepare decoder input and output by shifting answer sequences
decoder_input = answers_pad[:, :-1]
decoder_output = answers_pad[:, 1:]
decoder_output = np.expand_dims(decoder_output, -1)  # Add extra dim for sparse_categorical_crossentropy

# Vocabulary sizes
vocab_size_questions = len(tokenizer_questions.word_index) + 1
vocab_size_answers = len(tokenizer_answers.word_index) + 1

# Model parameters
embedding_dim = 256
lstm_units = 256

# Define encoder input
encoder_inputs = Input(shape=(max_len_questions,))
enc_emb = Embedding(vocab_size_questions, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Define decoder input
decoder_inputs = Input(shape=(max_len_answers - 1,))
dec_emb_layer = Embedding(vocab_size_answers, embedding_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_answers, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Compile model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 19, 256)   │    968,960 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 19)        │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 20, 256)   │    993,536 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 256),     │    525,312 │ embedding[0][0],  │
│                     │ (None, 256),      │            │ not_equal[0][0]   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 20, 256), │    525,312 │ embedding_1[0][0… │
│                     │ (None, 256),      │            │ lstm[0][1],       │
│                     │ (None, 256)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 20, 3881)  │    997,417 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,010,537 (15.30 MB)

 Trainable params: 4,010,537 (15.30 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(
    [questions_pad, decoder_input],
    decoder_output,
    batch_size=64,
    epochs=30,
    validation_split=0.2
)


Epoch 1/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 50s 973ms/step - loss: 7.5473 - val_loss: 5.8417
Epoch 2/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 45s 951ms/step - loss: 5.5055 - val_loss: 5.5052
Epoch 3/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 82s 954ms/step - loss: 5.1654 - val_loss: 5.3575
Epoch 4/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 83s 968ms/step - loss: 5.0288 - val_loss: 5.3297
Epoch 5/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 81s 958ms/step - loss: 4.9876 - val_loss: 5.3187
Epoch 6/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 83s 988ms/step - loss: 4.9340 - val_loss: 5.3221
Epoch 7/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 45s 954ms/step - loss: 4.9093 - val_loss: 5.2651
Epoch 8/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 82s 956ms/step - loss: 4.8687 - val_loss: 5.2325
Epoch 9/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 45s 959ms/step - loss: 4.7813 - val_loss: 5.1913
Epoch 10/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 82s 950ms/step - loss: 4.7526 - val_loss: 5.1522
Epoch 11/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - loss: 4.6956 - val_loss: 5.1339
Epoch 12/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 80s 9

In [6]:
# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)


In [7]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1 with only the start token.
    target_seq = np.array([tokenizer_answers.word_index['<start>']]).reshape(1, 1)

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_answers.index_word.get(sampled_token_index, '')

        if (sampled_word == '<end>' or sampled_word == '' or len(decoded_sentence.split()) > max_len_answers):
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

            # Update the target sequence (of length 1).
            target_seq = np.array([sampled_token_index]).reshape(1, 1)

            # Update states
            states_value = [h, c]

    return decoded_sentence.strip()


def chatbot_response(text):
    seq = tokenizer_questions.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len_questions, padding='post')
    response = decode_sequence(padded)
    return response


In [8]:
while True:
    input_text = input("You: ")
    if input_text.lower() in ['exit', 'quit']:
        break
    print("Bot:", chatbot_response(input_text))


You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Bot: i don't want to go to the new restaurant.
You: how are you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Bot: i don't want to go to the new restaurant.
You: I'm sorry, I didn't understand that. 

KeyboardInterrupt: Interrupted by user

In [9]:
def decode_sequence(input_seq, confidence_threshold=0.1):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.array([tokenizer_answers.word_index['<start>']]).reshape(1, 1)
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_answers.index_word.get(sampled_token_index, '')

        # Get the confidence (probability) of the predicted token
        confidence = output_tokens[0, -1, sampled_token_index]

        # If confidence is below threshold, use fallback response
        if confidence < confidence_threshold or sampled_word == '' or len(decoded_sentence.split()) > max_len_answers or sampled_word == '<end>':
            return "I'm sorry, I didn't understand that. Could you please rephrase?"

        decoded_sentence += ' ' + sampled_word
        target_seq = np.array([sampled_token_index]).reshape(1, 1)
        states_value = [h, c]

    return decoded_sentence.strip()


In [10]:
# Get user input (you can replace this with a fixed input to test)
input_text = input("You: ")

# Generate chatbot response
response = chatbot_response(input_text)

# Print response
print("Bot:", response)


You: Hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Bot: I'm sorry, I didn't understand that. Could you please rephrase?


In [11]:
# Get user input (you can replace this with a fixed input to test)
input_text = input("You: ")

# Generate chatbot response
response = chatbot_response(input_text)

# Print response
print("Bot:", response)


You: How are you ?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Bot: I'm sorry, I didn't understand that. Could you please rephrase?


In [12]:
def decode_sequence(input_seq, confidence_threshold=0.01):
    ...


In [13]:
print(f"Predicted word: {sampled_word}, Confidence: {confidence}")


NameError: name 'sampled_word' is not defined

In [14]:
# Get user input (you can replace this with a fixed input to test)
input_text = input("You: ")

# Generate chatbot response
response = chatbot_response(input_text)

# Print response
print("Bot:", response)


You: hi, how are you doing?
Bot: None


In [15]:
# Get user input (you can replace this with a fixed input to test)
input_text = input("You: ")

# Generate chatbot response
response = chatbot_response(input_text)

# Print response
print("Bot:", response)


You: what school do you go to?
Bot: None


In [16]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.array([tokenizer_answers.word_index['<start>']]).reshape(1, 1)
    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_answers.index_word.get(sampled_token_index, '')

        if sampled_word == '' or sampled_word == '<end>' or len(decoded_sentence.split()) > max_len_answers:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word
            target_seq = np.array([sampled_token_index]).reshape(1, 1)
            states_value = [h, c]

    return decoded_sentence.strip()


In [17]:
# Get user input (you can replace this with a fixed input to test)
input_text = input("You: ")

# Generate chatbot response
response = chatbot_response(input_text)

# Print response
print("Bot:", response)


You: what school do you go to?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Bot: i think i think you have to go to the new restaurant.


In [18]:
# Get user input (you can replace this with a fixed input to test)
input_text = input("You: ")

# Generate chatbot response
response = chatbot_response(input_text)

# Print response
print("Bot:", response)


You: hi, how are you doing?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Bot: i think you have to go to the new restaurant.


In [19]:
# Create dictionary for direct question-answer mapping
qa_dict = {q.lower().strip(): a for q, a in zip(df['question'], df['answer'])}

def chatbot_response_custom(text):
    query = text.lower().strip()
    if query in qa_dict:
        return qa_dict[query]
    else:
        # If no direct match, use model generated response
        seq = tokenizer_questions.texts_to_sequences([text])
        padded = pad_sequences(seq, maxlen=max_len_questions, padding='post')
        return decode_sequence(padded)


In [20]:
response = chatbot_response_custom(input_text)
print("Bot:", response)


Bot: i'm fine. how about yourself?


In [21]:
response = chatbot_response_custom(input_text)
print("Bot:", response)


Bot: i'm fine. how about yourself?


In [22]:
# Get user input (you can replace this with a fixed input to test)
input_text = input("You: ")

# Generate chatbot response
response = chatbot_response(input_text)

# Print response
print("Bot:", response)

You: hi, how are you doing?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Bot: i think you have to go to the new restaurant.


In [23]:
# Get user input (you can replace this with a fixed input to test)
input_text = input("You: ")

# Generate chatbot response
response = chatbot_response(input_text)

# Print response
print("Bot:", response)

You: Hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Bot: i don't want to go to the new restaurant.


In [24]:
input_text = input("You: ")
response = chatbot_response_custom(input_text)
print("Bot:", response)


You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Bot: i don't want to go to the new restaurant.


In [25]:
# Create dictionary for direct question-answer mapping
qa_dict = {q.lower().strip(): a for q, a in zip(df['question'], df['answer'])}

def chatbot_response_direct_only(text):
    query = text.lower().strip()
    # Return answer only if exact match found
    if query in qa_dict:
        return qa_dict[query]
    else:
        # If no match, say no answer available
        return "Sorry, I don't know how to respond to that."


In [26]:
input_text = input("You: ")
response = chatbot_response_direct_only(input_text)
print("Bot:", response)


You: hi, how are you doing?
Bot: i'm fine. how about yourself?


In [27]:
input_text = input("You: ")
response = chatbot_response_direct_only(input_text)
print("Bot:", response)

You: i'm pretty good. thanks for asking.
Bot: no problem. so how have you been?


In [28]:
qa_dict = {q.lower().strip(): a for q, a in zip(df['question'], df['answer'])}

def chatbot_response_direct_only(text):
    query = text.lower().strip()
    if query in qa_dict:
        return qa_dict[query]
    else:
        return "Sorry, I don't know how to respond to that."

print("Chatbot is ready! Type 'exit' to quit.")

while True:
    input_text = input("You: ")
    if input_text.lower() in ['exit', 'quit']:
        print("Bot: Goodbye!")
        break
    response = chatbot_response_direct_only(input_text)
    print("Bot:", response)


Chatbot is ready! Type 'exit' to quit.
You: i've been great. what about you?
Bot: i've been good. i'm in school right now.
You: what school do you go to?
Bot: i go to pcc.
You: good luck with school.
Bot: thank you very much.
You: exit
Bot: Goodbye!


In [29]:
# Save your trained model
model.save('chatbot_model.h5')

# Save tokenizers or other artifacts similarly (e.g., using pickle)
import pickle
with open('tokenizer_questions.pkl', 'wb') as f:
    pickle.dump(tokenizer_questions, f)

with open('tokenizer_answers.pkl', 'wb') as f:
    pickle.dump(tokenizer_answers, f)
